In [17]:
import requests
from bs4 import BeautifulSoup
# to use webdriver you need chromedriver.exe in the same folder as the .py script
from selenium import webdriver
import time
import os
import csv
from datetime import date
import random
from pikepdf import Pdf as PDF
from selenium.webdriver.common.by import By
# returns list of links of all pdf on a page

def get_soup_from_BeautifulSoup(url="https://www.google.com/search", params= None, *headers):
    response = requests.get(url, params=params, headers=headers )
    soup = BeautifulSoup(response.text, 'lxml')
    # with open("test.html", mode="w+", encoding='utf-8') as f:
    #     f.write(soup.prettify())
    #     f.close()
    return BeautifulSoup(response.text, 'lxml')
# using Beautiful Soup
# The main content of google search
# Selenium Implementation

def get_html_source(url):
    browser = webdriver.Chrome()
    # get web page
    browser.get(url)
    # area_select = browser.find_element_by_id(id_="areaselect")
    # execute script to scroll down the page
    time.sleep(40)
    show_more_button = browser.find_element(By.CSS_SELECTOR, 'button[ng-click="vm.pagginator.showmorepage()"]')
    while True:
        try:
            for i in range(0,200):
                browser.execute_script("window.scrollTo(0, 7/9*document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
                show_more_button.click()
        except:
            break
    time.sleep(10)
    # print(len(show_more_button))
    # for item in range(0,10):
    # with open("test.html", mode="w+", encoding='utf-8') as f:
    #     f.write(soup.prettify())
    #     f.close()
    # sleep for time
    return browser.page_source

# def get_google_url(params):
#     params_key, params_value = list(params.keys()),list(params.values())
#     url = f"https://www.google.com/search?"
#     for i in range(len(params)):
#         url = url + params_key[i] + "=" + str(params_value[i]).replace(" ", "+")+ "&"
#     return url


# --------------------------------------------------

In [18]:
soup = BeautifulSoup(get_html_source(url="https://www.bigbasket.com/cl/fruits-vegetables/"),"lxml")


In [27]:
product_list = soup.find_all(attrs={"qa": "product"})
final_data = []
pincode = soup.find_all("span", attrs={"ng-bind":"vm.user.currentAddress.address_display_name"})[0].text
for product_div in product_list:
    product_name = product_div.find_next(attrs={"qa": "product_name"})
    product_name = product_name.find_next("a").text

    weight = product_div.find_next("span",attrs={"ng-bind": "vm.selectedProduct.w"}).text
    price = product_div.find_next("span",attrs={"class":"discnt-price"}).text
    # price = price[3:]
    try:
        price = float(price)
    except :
        price = float(price.split(" ")[-1])
    if weight.split(" ")[-1].lower() == "g":
        price = price/float(weight.split(" ")[-2])*1000
        weight = "1 kg"
    elif weight.split(" ")[-1].lower() == "kg":
        price = price/float(weight.split(" ")[-2])
        weight = "1 kg"
    elif weight.split(" ")[-1].lower() == "combo":
        continue
    final_data.append({
        "name": product_name,
        "weight":weight,
        "price": price,
    })
print(len(final_data), "no. of elements stored")
print(final_data)

537 no. of elements stored
[{'name': 'Onion', 'weight': '1 kg', 'price': 38.38}, {'name': 'Tomato - Local', 'weight': '1 kg', 'price': 55.0}, {'name': 'Carrot - Orange', 'weight': '1 kg', 'price': 90.9}, {'name': 'Tomato - Hybrid', 'weight': '1 kg', 'price': 51.0}, {'name': 'Ladies Finger', 'weight': '1 kg', 'price': 89.0}, {'name': 'Cucumber', 'weight': '1 kg', 'price': 18.18}, {'name': 'Banana - Yelakki', 'weight': '1 kg', 'price': 46.46}, {'name': 'Coriander Leaves', 'weight': '1 kg', 'price': 70.7}, {'name': 'Orange - Nagpur, Regular', 'weight': '1 kg', 'price': 72.0}, {'name': 'Mushrooms - Button', 'weight': '1 pack', 'price': 42.42}, {'name': 'Beans - Haricot', 'weight': '1 kg', 'price': 52.0}, {'name': 'Capsicum - Green', 'weight': '1 kg', 'price': 90.9}, {'name': 'Palak - Cleaned, without roots', 'weight': '1 kg', 'price': 64.0}, {'name': 'Potato - New Crop', 'weight': '1 kg', 'price': 46.0}, {'name': 'Onion - Organically Grown', 'weight': '1 kg', 'price': 50.0}, {'name': 'Lemo

In [38]:
from openpyxl import Workbook, load_workbook
from datetime import date
excel_filename= f"{pincode}.xlsx"

try:
    wb = load_workbook(excel_filename)
except FileNotFoundError:
    wb = Workbook()
    wb.save(excel_filename)

ws = wb.active
maxrow=ws.max_row+1
pcol=dcol=ws.max_column+1

wcol=pcol+1
if dcol>2:
    print(ws.cell(row=1, column=dcol-2).value)
    if ws.cell(row=1, column=dcol-2).value == date.today().strftime("%d/%m/%Y"):
        pcol = pcol - 2
        dcol = dcol - 2
        wcol = wcol - 2
ws.cell(row=1, column=dcol).value = date.today().strftime("%d/%m/%Y")
ws.cell(row=2, column=pcol).value = "Price"
ws.cell(row=2, column=wcol).value = "Weight/pc(s)"
for index_1, dict in enumerate(final_data):
    c=1
    for i in range(maxrow-1):
        if ws.cell(row=i+1, column=1).value == dict["name"]:
            c=0
            ws.cell(row=i+1, column=pcol).value = dict["price"]
            ws.cell(row=i+1, column=wcol).value = dict["weight"]
    if c==1:
        print(ws.max_row)
        ws.cell(row=ws.max_row+1, column=1).value = dict["name"]
        ws.cell(row=ws.max_row, column=pcol).value = dict["price"]
        ws.cell(row=ws.max_row, column=wcol).value = dict["weight"]
        
wb.save(excel_filename)

04/01/2022


range(0, 37)
